# ML Model Training and Evaluation Assignment

## 1. Import Libraries

In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'seaborn'

## 2. Load Dataset

In [ ]:

df = pd.read_csv("breast_cancer_data.csv")
df.head()


## 3. Preprocessing

In [ ]:

X = df.drop("target", axis=1)
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 4. Train Multiple Models

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC()
}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }

pd.DataFrame(results).T


## 5. Hyperparameter Tuning - Random Forest (GridSearchCV)

In [ ]:

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1')
grid.fit(X_train_scaled, y_train)

best_rf = grid.best_estimator_
y_pred_rf = best_rf.predict(X_test_scaled)
print("Best Random Forest (GridSearchCV):")
print(classification_report(y_test, y_pred_rf))


## 6. Hyperparameter Tuning - SVM (RandomizedSearchCV)

In [ ]:

from scipy.stats import uniform

param_dist = {
    'C': uniform(0.1, 10),
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'rbf']
}

random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=5, scoring='f1', random_state=42)
random_search.fit(X_train_scaled, y_train)

best_svm = random_search.best_estimator_
y_pred_svm = best_svm.predict(X_test_scaled)
print("Best SVM (RandomizedSearchCV):")
print(classification_report(y_test, y_pred_svm))


## 7. Final Model Comparison

In [ ]:

final_results = {
    "Logistic Regression": results["Logistic Regression"],
    "Tuned Random Forest": {
        "Accuracy": accuracy_score(y_test, y_pred_rf),
        "Precision": precision_score(y_test, y_pred_rf),
        "Recall": recall_score(y_test, y_pred_rf),
        "F1-Score": f1_score(y_test, y_pred_rf)
    },
    "Tuned SVM": {
        "Accuracy": accuracy_score(y_test, y_pred_svm),
        "Precision": precision_score(y_test, y_pred_svm),
        "Recall": recall_score(y_test, y_pred_svm),
        "F1-Score": f1_score(y_test, y_pred_svm)
    }
}

pd.DataFrame(final_results).T
